In [33]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from xgboost import plot_importance


%matplotlib inline

# EDA

In [34]:
import seaborn as sns
import matplotlib.pyplot as plt

# 모델링

In [36]:
# drop_df = df.drop(['Unnamed: 0', 'gameId','redWins',
#                    "blueFirstTowerLane",'blueDragonType','redFirstTowerLane','redDragonType'], axis=1)
# drop_df
df = pd.read_csv("/Users/ichang-ug/Documents/GitHub/esports_YSAL/LOL/Data/CL15_fix.csv")

In [37]:
df.dtypes.unique()

array([dtype('int64')], dtype=object)

In [38]:
def train_test(df):
  y_train = df['blueWins']
  x_train = df.drop(['blueWins'], axis=1)
  return x_train, y_train

In [39]:
x_train, y_train = train_test(df)

val_size=0.3

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = val_size, random_state=42)

In [40]:
x_train.columns

Index(['blueTotalGolds', 'blueTotalLevel', 'blueTotalMinionKills',
       'blueTotalJungleMinionKills', 'blueTotalDamageToChampion',
       'blueFirstBlood', 'blueKill', 'blueDeath', 'blueAssist',
       'blueWardPlaced', 'blueWardKills', 'blueFirstTower',
       'blueFirstInhibitor', 'blueTowerKills', 'blueMidTowerKills',
       'blueTopTowerKills', 'blueBotTowerKills', 'blueInhibitor',
       'blueFirstDragon', 'blueDragon', 'blueRiftHeralds', 'redTotalGolds',
       'redTotalLevel', 'redTotalMinionKills', 'redTotalJungleMinionKills',
       'redTotalDamageToChampion', 'redFirstBlood', 'redKill', 'redDeath',
       'redAssist', 'redWardPlaced', 'redWardKills', 'redFirstTower',
       'redFirstInhibitor', 'redTowerKills', 'redMidTowerKills',
       'redTopTowerKills', 'redBotTowerKills', 'redInhibitor',
       'redFirstDragon', 'redDragon', 'redRiftHeralds', 'matchMin',
       'blueWATER_DRAGON', 'blueFIRE_DRAGON', 'blueAIR_DRAGON',
       'blueEARTH_DRAGON', 'blueCHEMTECH_DRAGON', 'b

## Xgboost

파라미터 튜닝

In [18]:
from bayes_opt import BayesianOptimization

In [19]:
# 탐색 대상 함수 (XGBRegressor)
def XGB_cv(max_depth,learning_rate, n_estimators, gamma
            ,min_child_weight, subsample
            ,colsample_bytree, silent=True, nthread=-1):

    # 모델 정의
    model = xgb.XGBClassifier(max_depth=int(max_depth),
                            learning_rate=learning_rate,
                            n_estimators=int(n_estimators),
                            gamma=gamma,
                            min_child_weight=min_child_weight,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree, 
                            nthread=nthread
                            )
    # 모델 훈련
    model.fit(x_train, y_train)

    # 예측값 출력
    y_pred= model.predict(x_valid)

    return f1_score(y_valid, y_pred)

In [20]:
pbounds = {'max_depth': (3, 7),
                'learning_rate': (0.01, 0.3),
                'n_estimators': (20,100),
                'gamma': (0, 100),
                'min_child_weight': (0, 3),
                'subsample': (0.5, 1),
                'colsample_bytree' :(0.2, 1)
                }

bo=BayesianOptimization(f=XGB_cv, pbounds=pbounds, verbose=2, random_state=1 )    

bo.maximize(init_points=2, n_iter=100)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------


NameError: name 'x_train' is not defined

## Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier


In [22]:
def bo_params_rf(max_samples,n_estimators,max_features, max_depth):
    
    params = {
        'max_samples': max_samples,
        'max_features':max_features,
        'max_depth':int(max_depth),
        'n_estimators':int(n_estimators)
    }
    clf = RandomForestClassifier(**params)
    
    clf.fit(x_train, y_train)

    # 예측값 출력
    y_pred= clf.predict(x_valid)

    return f1_score(y_valid, y_pred)

In [23]:
pbounds = {'max_samples':(0.5,1),
        'max_features':(0.5,1),
        'max_depth' : (3, 7),
        'n_estimators':(10,200)}

rf_bo = BayesianOptimization(bo_params_rf, pbounds=pbounds, verbose=2, random_state=1)
rf_bo.maximize(init_points=2, n_iter=100)

|   iter    |  target   | max_depth | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------------------


NameError: name 'x_train' is not defined

In [24]:
print(bo.max)
print(rf_bo.max)

{}
{}


## 최종 모델

Xgboost

In [41]:
xgb_params = {'colsample_bytree': 0.7399756367368562, 'gamma': 23.60157571112307, 'learning_rate': 0.1641944292221702, 
          'max_depth': 6, 'min_child_weight': 1.3577908241933232, 'n_estimators': 36, 'subsample': 0.8954956520260448}

xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7399756367368562, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=23.60157571112307, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1641944292221702, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=1.3577908241933232, missing=nan,
              monotone_constraints=None, n_estimators=36, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

RF

In [95]:
rf_params = {'max_depth': 5, 'max_features': 0.5152282117692119, 
          'max_samples': 0.6447967093611578, 'n_estimators': 133}
rf_model = RandomForestClassifier(**rf_params)
rf_model.fit(x_train, y_train)

RandomForestClassifier(max_depth=5, max_features=0.5152282117692119,
                       max_samples=0.6447967093611578, n_estimators=133)

# 다른 데이터에 적용

In [42]:
# cl10 = pd.read_csv("/content/drive/MyDrive/YSAL/2023-1/LOL/CL10_fix.csv")
cl15 = pd.read_csv("/Users/ichang-ug/Documents/GitHub/esports_YSAL/LOL/Data/CL15_fix.csv")

In [43]:
x_cl15 = cl15.drop("blueWins", axis=1)
y_cl15 = cl15["blueWins"]
x_cl15 = cl15.drop("blueWins", axis=1)
y_cl15 = cl15

## Cl_10

xgboost

In [47]:
y_pred = xgb_model.predict(x_cl15)
print("F1 score: ", f1_score(y_cl15, y_pred))
print("Accuracy: %.2f%%" % (accuracy_score(y_cl15, y_pred) * 100.0))

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and binary targets

Rf

In [1]:
y_pred = rf_model.predict(x_cl10)
print("F1 sore: ", f1_score(y_cl15,y_pred)  )
print("Accuracy: %.2f%%" % (accuracy_score(y_cl10, y_pred) * 100.0))

NameError: name 'rf_model' is not defined

In [49]:
cl15.columns

Index(['blueWins', 'blueTotalGolds', 'blueTotalLevel', 'blueTotalMinionKills',
       'blueTotalJungleMinionKills', 'blueTotalDamageToChampion',
       'blueFirstBlood', 'blueKill', 'blueDeath', 'blueAssist',
       'blueWardPlaced', 'blueWardKills', 'blueFirstTower',
       'blueFirstInhibitor', 'blueTowerKills', 'blueMidTowerKills',
       'blueTopTowerKills', 'blueBotTowerKills', 'blueInhibitor',
       'blueFirstDragon', 'blueDragon', 'blueRiftHeralds', 'redTotalGolds',
       'redTotalLevel', 'redTotalMinionKills', 'redTotalJungleMinionKills',
       'redTotalDamageToChampion', 'redFirstBlood', 'redKill', 'redDeath',
       'redAssist', 'redWardPlaced', 'redWardKills', 'redFirstTower',
       'redFirstInhibitor', 'redTowerKills', 'redMidTowerKills',
       'redTopTowerKills', 'redBotTowerKills', 'redInhibitor',
       'redFirstDragon', 'redDragon', 'redRiftHeralds', 'matchMin',
       'blueWATER_DRAGON', 'blueFIRE_DRAGON', 'blueAIR_DRAGON',
       'blueEARTH_DRAGON', 'blueCHEMTECH